In [1]:
import textrazor
#Configuración API TextRazor
textrazor.api_key = "5d308b9cfb4a9e9a0a1c36eb8885602f617d236465437fe5cec90a0c"
client = textrazor.TextRazor(extractors=["entities", "topics"])
client.set_classifiers(["textrazor_newscodes"])

In [4]:
from pandas import json_normalize
import pandas as pd

tripletasOM = []
#Leer CVS de OER (Recursos Educativos Abiertos)
datosCSV = pd.read_csv("newdata.csv")

for index, row in datosCSV.iterrows():
    #Identificación de entidades en el Resumen de un OER
    response = client.analyze(row['RESUMEN']) 
    
    for entity in response.entities():
        #Selección de las entidades mas relevantes con score +0.3
        if(entity.relevance_score > 0.3):
            # Sujeto, Predicado, Objeto 
            S1 = "<http://data.utpl.edu.ec/oer/"+row['TESIS']+"> "
            P1 = "<http://schema.org/mentions> "
            O1 = "<"+entity.wikipedia_link.replace("wikipedia","dbpedia").replace("/wiki","/resource")+"> ."
            S2 = "<"+entity.wikipedia_link.replace("wikipedia","dbpedia").replace("/wiki","/resource")+"> "
            P2 = "<http://www.w3.org/2000/01/rdf-schema#label> "
            O2 = "\""+entity.id+"\" ."
            
            tripleta1 = (S1, P1, O1)
            tripleta2 = (O2, P2, O2)
            
            tripletasOM.append(tipleta1)
            tripletasOM.append(tipleta2)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF, CSV, TSV
from pandas import json_normalize
import pandas as pd
sparql = SPARQLWrapper("http://es.dbpedia.org/sparql")

In [ ]:
for item in tripletasOM:
    #extraer Uri de la mention
    if ("<" in item[2]):
        mention = i[2].replace(" .","")
        #Consulta SPARQL para extraer dct:subject y skos:broader
        sparql.setQuery("""
            PREFIX dct: <http://purl.org/dc/terms/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            SELECT * WHERE {
            values ?nodo {"""+mention+"""}
            values ?p {dct:subject} 
            ?nodo ?p ?o.
            values ?p1 {rdfs:label}
            ?o ?p1 ?l .
            values ?p2 {skos:broader}
            ?o ?p2 ?s.
            ?s ?p1 ?ss. 
            }
            """)
        sparql.setReturnFormat(JSON)
        r = sparql.query().convert()
        results = json_normalize(r["results"]["bindings"])
        
        for index, row in results.iterrows():
            t1 = (row['nodo.value'], row['p.value'], row['o.value'])
            t2 = (row['o.value'], row['p1.value'], row['l.value'])
            t3 = (row['o.value'], row['p2.value'], row['s.value'])
            t4 = (row['s.value'], row['p1.value'], row['ss.value'])
            if (t1 in tripletasOM) == False and (t2 in tripletasOM) == False :
                tripletasOM.append(t1)
                tripletasOM.append(t2)
            tripletasOM.append(t3)
            tripletasOM.append(t4)